In [3]:
%%capture

! pip install transformers
! pip install accelerate -U
! pip install datasets

! pip install evaluate
! pip install sacrebleu

In [4]:
from datasets import load_dataset

dataset = load_dataset("XvKuoMing/casings")

In [5]:
from transformers import AutoTokenizer

checkpoint = "ai-forever/ruT5-base"
# checkpoint = "sn4kebyt3/ru-bart-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          add_prefix_space=True,
                                          device_map="auto")

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [18]:
import random


MAX_LENGTH = 24

def preprocess(batch):

    targs = {'max_length':MAX_LENGTH,
             'truncation':True,
             'is_split_into_words':True}
    lemmas = []
    tokens = []
    for n, sub_lemmas in enumerate(batch['lemmas']):
        if len(sub_lemmas) > MAX_LENGTH or not bool(sub_lemmas):
            continue
        sub_tokens = batch['tokens'][n]
        while sub_lemmas[-1] == '.':
            sub_lemmas = sub_lemmas[:-1]
            sub_tokens = sub_tokens[:-1]
#         lemmas.append(sub_lemmas)
#         tokens.append(sub_tokens)
        # corrupting 
        last = 0
        step = 4
        corrupted = []
        for _ in range(len(sub_lemmas[::step])):
            sent_piece = sub_lemmas[last:last+step]
            corrupted.append(
                sub_lemmas[:last] + random.sample(sent_piece, len(sent_piece)) + sub_lemmas[last+step:]
            )
            last += step
#         lemmas.append(random.choice(corrupted))
#         tokens.append(sub_tokens)
         # уберем базовые предлоги и запятые
        no_preps_lemmas = [lemma for lemma in random.choice(corrupted)
                          if lemma not in [',', 'в', 'от', 'о', 'к', 'по', 'за', 'на']]
        if no_preps_lemmas:
            lemmas.append(no_preps_lemmas)
            tokens.append(sub_tokens)
        else: # если предложение состояло из мусора
            continue
    if lemmas:
        return tokenizer(lemmas,
                         text_target=tokens,
                         **targs)
    else: # может оказаться так что во всем батче не будут только длинные предложения
        return {'input_ids':[],
               'attention_mask':[],
               'labels':[]}

In [7]:
tokenized_data = dataset.map(preprocess,
                            batched=True,
                            batch_size=8,
                            remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/61461 [00:00<?, ? examples/s]

Map:   0%|          | 0/15366 [00:00<?, ? examples/s]

In [8]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 61037
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15280
    })
})

In [9]:
# немножко увеличим наш датасет предложениями из совершенно другого корпуса
from datasets import load_dataset

additional_dataset = load_dataset("universal_dependencies", "ru_syntagrus")

Generating train split:   0%|          | 0/48814 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6584 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6491 [00:00<?, ? examples/s]

In [19]:
add_data_tokens = additional_dataset.map(preprocess,
                                        batched=True,
                                        batch_size=8,
                                        remove_columns=additional_dataset['train'].column_names)

Map:   0%|          | 0/48814 [00:00<?, ? examples/s]

Map:   0%|          | 0/6584 [00:00<?, ? examples/s]

Map:   0%|          | 0/6491 [00:00<?, ? examples/s]

In [20]:
add_data_tokens

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 37708
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5096
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5006
    })
})

In [21]:
from datasets import concatenate_datasets, DatasetDict

vectors = concatenate_datasets([tokenized_data['train'],
                                tokenized_data['test'],
                                add_data_tokens['test'],
                                add_data_tokens['validation'],
                               ]).train_test_split(0.2)
vectors

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 69135
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 17284
    })
})

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=checkpoint)

2024-05-15 09:34:07.966539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 09:34:07.966684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 09:34:08.108670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [23]:
import evaluate

bleu = evaluate.load("sacrebleu")
# meteor = evaluate.load('meteor')
chrf = evaluate.load("chrf")

In [24]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
#     meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_result = chrf.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": bleu_result["score"],
#               "meteor": meteor_result['meteor'],
              "chrf": chrf_result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [25]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

gen_config = deepcopy(model.generation_config)
gen_config.update(
    max_length=MAX_LENGTH,
)
gen_config.validate()

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-rebuilder-v2",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    generation_config=gen_config,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=vectors['train'],
    eval_dataset=vectors['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Gen Len
1,0.446300,0.320490,78.661300,90.200400,13.296700
2,0.240000,0.237883,84.276300,92.789800,13.394000
3,0.118000,0.220571,86.013400,93.657300,13.400700


TrainOutput(global_step=12963, training_loss=0.35032713222878864, metrics={'train_runtime': 4306.557, 'train_samples_per_second': 48.16, 'train_steps_per_second': 3.01, 'total_flos': 5563453560606720.0, 'train_loss': 0.35032713222878864, 'epoch': 3.0})

In [28]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

events.out.tfevents.1715765690.19a19999c1eb.34.0:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/6.65k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/XvKuoMing/t5-rebuilder-v2/commit/37763dbeb90152dfcab5e7083b6520a7b8c826d8', commit_message='End of training', commit_description='', oid='37763dbeb90152dfcab5e7083b6520a7b8c826d8', pr_url=None, pr_revision=None, pr_num=None)